In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-funnel-olist/olist_closed_deals_dataset.csv
/kaggle/input/marketing-funnel-olist/olist_marketing_qualified_leads_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


## 사용 도구
### SQL
- sqlite3 라이브러리를 사용해 가상 메모리 DB를 생성
- Input의 CSV 데이터를 SQLite 테이블로 변환하여 SQL 쿼리 실행
- 데이터 전처리 및 집계 작업 수행



### Tableau
- SQL 분석 결과를 기반으로 시각화 진행
- 퍼널 분석, 유입 채널 비교, 업종별 전환율 등을 시각적으로 표현
- 최종 결과를 링크 형태로 공유

### Python
- pandas, sklearn 등을 활용하여 데이터 분석 기법 적용
- MQL 리드 클러스터링(KMeans) 및 PCA 기반 시각화
- 각 군집의 특성을 분석해 마케팅 전략 제안 도출

In [2]:
import pandas as pd
import sqlite3

In [3]:
# csv 파일 불러오기
mql = pd.read_csv("/kaggle/input/marketing-funnel-olist/olist_marketing_qualified_leads_dataset.csv")
deals = pd.read_csv("/kaggle/input/marketing-funnel-olist/olist_closed_deals_dataset.csv", 
                    # 결측치 비율 90% 이상 컬럼 제거
                    usecols = lambda col:col not in ['has_compnay','has_gtin','average_stock','declared_product_catalog_size', 'declared_monthly_revenue' 
                                                     
])
orders = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv")
sellers = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv")

# 데이터프레임을 SQL 테이블로 저장
conn = sqlite3.connect(":memory:")

mql.to_sql("mql",conn, index = False)
deals.to_sql("deals",conn, index = False)
orders.to_sql("orders",conn, index = False)
sellers.to_sql("sellers",conn, index = False)

3095

In [4]:
query = '''
SELECT * FROM mql LIMIT 5;
'''

pd.read_sql(query,conn)

,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [5]:
query = '''
SELECT * FROM deals LIMIT 5;
'''

pd.read_sql(query,conn)

,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,business_type
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,None,reseller
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,None,reseller
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,None,reseller
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,None,None,reseller
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,None,manufacturer


In [6]:
# 채널별 유입

query = '''
SELECT origin, COUNT(*) as count
FROM mql
GROUP BY origin
ORDER BY count DESC
'''

pd.read_sql(query, conn)

,origin,count
0,organic_search,2296
1,paid_search,1586
2,social,1350
3,unknown,1099
4,direct_traffic,499
5,email,493
6,referral,284
7,other,150
8,display,118
9,other_publicities,65
